# 🧠 SV-NSDE: Semantic Volatility-Modulated Neural SDE

**Modeling Crisis Dynamics: Distinguishing Panic from Trends in COVID-19 Information Diffusion**

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/YOUR_USERNAME/YOUR_REPO/blob/main/notebooks/SV_NSDE_Colab.ipynb)

---

## 📋 Notebook 内容

1. **环境设置** - 安装依赖
2. **数据准备** - 生成/加载数据
3. **模型架构** - SV-NSDE核心组件
4. **训练模型** - 完整训练流程
5. **评估对比** - 与基线模型对比
6. **波动率分析** - 识别恐慌事件
7. **可视化** - SDE轨迹与强度

---

In [3]:
# mount google drive to /content/drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [10]:
! ls drive/My\ Drive/WeiboCOV 

'Copy of 2019-12.csv'  'Copy of 2020-05.csv'  'Copy of 2020-10.csv'
'Copy of 2020-01.csv'  'Copy of 2020-06.csv'  'Copy of 2020-11.csv'
'Copy of 2020-02.csv'  'Copy of 2020-07.csv'  'Copy of 2020-12.csv'
'Copy of 2020-03.csv'  'Copy of 2020-08.csv'  'Copy of user.csv'
'Copy of 2020-04.csv'  'Copy of 2020-09.csv'


## 1️⃣ 环境设置

In [ ]:
# 检查是否在 Colab 环境
import sys
IN_COLAB = 'google.colab' in sys.modules

if IN_COLAB:
    print("🚀 Running in Google Colab")
    # 克隆仓库
    !git clone https://github.com/trumpool/Research.git sv_nsde_repo
    %cd sv_nsde_repo
    
    # 安装依赖
    !pip install -q torch numpy transformers tqdm matplotlib seaborn
    !pip install -e .
else:
    print("💻 Running locally")
    # 本地运行时，确保在项目根目录
    import os
    if os.path.exists('../src'):
        sys.path.insert(0, '../src')

In [ ]:
# 检查 GPU
import torch

device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"🖥️ Device: {device}")

if device == "cuda":
    print(f"   GPU: {torch.cuda.get_device_name(0)}")
    print(f"   Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")

In [ ]:
# 导入所需模块
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm.auto import tqdm

# 设置绘图风格
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette("husl")

print("✓ Libraries imported")

## 2️⃣ 数据准备

In [ ]:
# 导入 SV-NSDE 模块
from sv_nsde import (
    SVNSDELite,
    generate_synthetic_weibo_data,
    WeiboCOVLoader,
    CascadeDataset,
    get_baseline,
    Evaluator,
)

print("✓ SV-NSDE modules imported")

In [ ]:
# 生成合成数据 (无需下载)
import os
os.makedirs("data", exist_ok=True)

print("📊 Generating synthetic Weibo-COV data...")
df = generate_synthetic_weibo_data(
    n_cascades=300,  # 级联数量
    output_path="data/synthetic_weibo.csv",
    seed=42
)

print(f"✓ Generated {len(df):,} events")
df.head()

In [ ]:
# 加载并构建级联
loader = WeiboCOVLoader("data/synthetic_weibo.csv")
loader.load()
cascades = loader.build_cascades(min_size=5, max_size=100)

# 获取统计信息
stats = loader.get_statistics()
print(f"\n📈 Dataset Statistics:")
print(f"   Cascades: {stats['num_cascades']}")
print(f"   Total events: {stats['total_events']}")
print(f"   Avg size: {stats['size_mean']:.1f} ± {stats['size_std']:.1f}")
print(f"   Max size: {stats['size_max']}")

In [ ]:
# 按时间分割数据
train_cascades, val_cascades, test_cascades = loader.split_by_time(
    train_end="2020-02-29",  # 爆发期
    val_end="2020-03-31"     # 平台期
)

print(f"\n📂 Data Split:")
print(f"   Train: {len(train_cascades)} cascades (outbreak period)")
print(f"   Val:   {len(val_cascades)} cascades (plateau period)")
print(f"   Test:  {len(test_cascades)} cascades (decline period)")

In [ ]:
# 可视化级联大小分布
sizes = [c.size for c in cascades]

fig, axes = plt.subplots(1, 2, figsize=(12, 4))

# 直方图
axes[0].hist(sizes, bins=30, edgecolor='white', alpha=0.7)
axes[0].set_xlabel('Cascade Size')
axes[0].set_ylabel('Frequency')
axes[0].set_title('Distribution of Cascade Sizes')
axes[0].axvline(np.mean(sizes), color='red', linestyle='--', label=f'Mean: {np.mean(sizes):.1f}')
axes[0].legend()

# Log-log (检查幂律分布)
axes[1].hist(sizes, bins=np.logspace(0, np.log10(max(sizes)), 30), edgecolor='white', alpha=0.7)
axes[1].set_xscale('log')
axes[1].set_yscale('log')
axes[1].set_xlabel('Cascade Size (log)')
axes[1].set_ylabel('Frequency (log)')
axes[1].set_title('Power-Law Check (Log-Log)')

plt.tight_layout()
plt.show()

## 3️⃣ 模型架构

### SV-NSDE 核心组件

**耦合SDE系统 (Equation 3):**

$$dz(t) = \mu_\theta(z,t)dt + \sqrt{v(t)} \odot dW_z(t) + J_\phi(z(t^-), x_i)dN(t)$$

$$dv(t) = \kappa(\theta - v(t))dt + \xi\sqrt{v(t)} \odot dW_v(t)$$

**双通道强度函数 (Equation 4):**

$$\lambda(t) = \text{Softplus}\left(\underbrace{w_{tr}^\top m(z(t))}_{\text{Trend}} + \underbrace{w_{vol}^\top g(v(t))}_{\text{Volatility}} + \mu_{base}\right)$$

In [ ]:
# 配置参数
D_LATENT = 32   # 潜在空间维度
D_HIDDEN = 64   # 隐藏层维度

print(f"⚙️ Model Configuration:")
print(f"   d_latent: {D_LATENT}")
print(f"   d_hidden: {D_HIDDEN}")
print(f"   device: {device}")

In [ ]:
# 创建 SV-NSDE 模型
model = SVNSDELite(
    d_input=D_LATENT,
    d_latent=D_LATENT,
    d_hidden=D_HIDDEN,
).to(device)

# 打印模型结构
print("\n🧠 SV-NSDE Architecture:")
print("=" * 50)
n_params = sum(p.numel() for p in model.parameters())
print(f"Total parameters: {n_params:,}")
print("\nComponents:")
for name, module in model.named_children():
    params = sum(p.numel() for p in module.parameters())
    print(f"  {name}: {params:,} params")

In [ ]:
# 查看 SDE 组件
print("\n📐 Neural Heston SDE:")
print("  Drift: μ_θ(z, t) - neural network")
print("  Diffusion: √v(t) - stochastic volatility")
print("  Jump: J_φ(z, x) - event-driven updates")
print(f"\n  CIR parameters:")
print(f"    κ (mean reversion): {model.sde.sde_func.kappa.item():.4f}")
print(f"    θ (long-term var):  {model.sde.sde_func.theta.item():.4f}")
print(f"    ξ (vol of vol):     {model.sde.sde_func.xi.item():.4f}")

In [ ]:
# 可视化 SDE 路径采样 (无事件)
from sv_nsde.sde import NeuralHestonSDE

# 创建2维SDE用于可视化
sde_2d = NeuralHestonSDE(d_latent=2).to(device)

# 采样多条路径
with torch.no_grad():
    z_paths, v_paths, times = sde_2d.sample_paths(
        n_paths=10,
        T=1.0,
        dt=0.01,
        device=device
    )

z_paths = z_paths.cpu().numpy()
v_paths = v_paths.cpu().numpy()

# 绘图
fig, axes = plt.subplots(1, 3, figsize=(15, 4))

# z(t) 轨迹
for i in range(z_paths.shape[1]):
    axes[0].plot(times, z_paths[:, i, 0], alpha=0.7)
axes[0].set_xlabel('Time')
axes[0].set_ylabel('z(t) dim 0')
axes[0].set_title('Semantic Trajectory z(t)')

# v(t) 波动率
for i in range(v_paths.shape[1]):
    axes[1].plot(times, v_paths[:, i, 0], alpha=0.7)
axes[1].set_xlabel('Time')
axes[1].set_ylabel('v(t) dim 0')
axes[1].set_title('Volatility Process v(t)')

# 2D 相空间
for i in range(min(5, z_paths.shape[1])):
    axes[2].plot(z_paths[:, i, 0], z_paths[:, i, 1], alpha=0.7)
    axes[2].scatter(z_paths[0, i, 0], z_paths[0, i, 1], s=50, marker='o')  # 起点
    axes[2].scatter(z_paths[-1, i, 0], z_paths[-1, i, 1], s=50, marker='x')  # 终点
axes[2].set_xlabel('z dim 0')
axes[2].set_ylabel('z dim 1')
axes[2].set_title('2D Phase Space')

plt.tight_layout()
plt.show()

## 4️⃣ 训练模型

In [ ]:
# 准备训练数据 (使用随机嵌入代替BERT)
print("📦 Preparing training data...")

train_embeddings = {}
val_embeddings = {}

for c in train_cascades:
    train_embeddings[c.cascade_id] = torch.randn(c.size, D_LATENT)

for c in val_cascades:
    val_embeddings[c.cascade_id] = torch.randn(c.size, D_LATENT)

print(f"✓ Train embeddings: {len(train_embeddings)}")
print(f"✓ Val embeddings: {len(val_embeddings)}")

In [ ]:
# 训练配置
LEARNING_RATE = 1e-3
BATCH_SIZE = 8
NUM_EPOCHS = 20  # Colab上可以增加到50-100

print(f"\n🎯 Training Configuration:")
print(f"   Learning rate: {LEARNING_RATE}")
print(f"   Batch size: {BATCH_SIZE}")
print(f"   Epochs: {NUM_EPOCHS}")

In [ ]:
# 简单训练循环
from torch.optim import AdamW
from torch.optim.lr_scheduler import CosineAnnealingLR

# 重新创建模型
model = SVNSDELite(
    d_input=D_LATENT,
    d_latent=D_LATENT,
    d_hidden=D_HIDDEN,
).to(device)

optimizer = AdamW(model.parameters(), lr=LEARNING_RATE, weight_decay=0.01)
scheduler = CosineAnnealingLR(optimizer, T_max=NUM_EPOCHS, eta_min=LEARNING_RATE * 0.01)

train_losses = []
val_losses = []

print("\n🚀 Starting training...\n")

for epoch in range(1, NUM_EPOCHS + 1):
    model.train()
    epoch_loss = 0
    n_batches = 0
    
    # 训练
    for cascade in train_cascades:
        if cascade.size < 3:
            continue
            
        event_times = cascade.event_times.to(device)
        event_marks = train_embeddings[cascade.cascade_id].to(device)
        
        # 归一化时间
        T = event_times.max().item()
        if T > 0:
            event_times = event_times / T
        
        # 前向传播
        loss_dict = model.compute_loss(event_times, event_marks, T=1.0)
        loss = loss_dict['loss']
        
        # 反向传播
        optimizer.zero_grad()
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()
        
        epoch_loss += loss.item()
        n_batches += 1
    
    avg_train_loss = epoch_loss / max(n_batches, 1)
    train_losses.append(avg_train_loss)
    
    # 验证
    model.eval()
    val_loss = 0
    n_val = 0
    
    with torch.no_grad():
        for cascade in val_cascades:
            if cascade.size < 3:
                continue
            
            event_times = cascade.event_times.to(device)
            event_marks = val_embeddings[cascade.cascade_id].to(device)
            
            T = event_times.max().item()
            if T > 0:
                event_times = event_times / T
            
            loss_dict = model.compute_loss(event_times, event_marks, T=1.0)
            val_loss += loss_dict['loss'].item()
            n_val += 1
    
    avg_val_loss = val_loss / max(n_val, 1)
    val_losses.append(avg_val_loss)
    
    scheduler.step()
    
    # 打印进度
    if epoch % 5 == 0 or epoch == 1:
        print(f"Epoch {epoch:3d}/{NUM_EPOCHS}: train_loss={avg_train_loss:.2f}, val_loss={avg_val_loss:.2f}")

print("\n✓ Training complete!")

In [ ]:
# 可视化训练曲线
fig, ax = plt.subplots(figsize=(10, 5))

ax.plot(range(1, len(train_losses)+1), train_losses, label='Train Loss', marker='o', markersize=3)
ax.plot(range(1, len(val_losses)+1), val_losses, label='Val Loss', marker='s', markersize=3)
ax.set_xlabel('Epoch')
ax.set_ylabel('Loss')
ax.set_title('Training Curves')
ax.legend()
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 5️⃣ 评估与基线对比

In [ ]:
# 创建基线模型
print("🏗️ Initializing baseline models...")

baselines = {
    "SV-NSDE (ours)": model,
    "RMTPP": get_baseline("rmtpp", d_input=D_LATENT, d_latent=D_LATENT, d_hidden=D_HIDDEN),
    "Neural Hawkes": get_baseline("neural_hawkes", d_input=D_LATENT, d_latent=D_LATENT, d_hidden=D_HIDDEN),
    "Latent ODE": get_baseline("latent_ode", d_input=D_LATENT, d_latent=D_LATENT, d_hidden=D_HIDDEN),
    "Neural Jump SDE": get_baseline("neural_jump_sde", d_input=D_LATENT, d_latent=D_LATENT, d_hidden=D_HIDDEN),
}

for name, m in baselines.items():
    m.to(device)
    params = sum(p.numel() for p in m.parameters())
    print(f"  {name}: {params:,} params")

In [ ]:
# 准备测试数据
test_embeddings = {}
for c in test_cascades:
    test_embeddings[c.cascade_id] = torch.randn(c.size, D_LATENT)

print(f"📊 Test data: {len(test_cascades)} cascades")

In [ ]:
# 运行评估
print("\n📏 Running evaluation...\n")

evaluator = Evaluator(device=device)

results = {}
for name, m in baselines.items():
    print(f"Evaluating {name}...")
    try:
        metrics = evaluator.evaluate_model(
            m, 
            test_cascades[:30],  # 使用子集加快速度
            test_embeddings,
            verbose=False
        )
        results[name] = metrics
        print(f"  LL={metrics.log_likelihood:.2f}, Cosine={metrics.semantic_cosine:.4f}")
    except Exception as e:
        print(f"  Error: {e}")

In [ ]:
# 可视化对比结果
import pandas as pd

# 创建结果表格
rows = []
for name, metrics in results.items():
    rows.append({
        'Model': name,
        'Log-Likelihood': metrics.log_likelihood,
        'Semantic Cosine': metrics.semantic_cosine,
        'Time RMSE': metrics.time_rmse,
    })

df_results = pd.DataFrame(rows)
print("\n📊 Comparison Results:")
print(df_results.to_string(index=False))

In [ ]:
# 柱状图对比
fig, axes = plt.subplots(1, 2, figsize=(12, 5))

models = list(results.keys())
colors = plt.cm.Set2(np.linspace(0, 1, len(models)))

# Log-Likelihood
lls = [results[m].log_likelihood for m in models]
valid_lls = [(m, ll) for m, ll in zip(models, lls) if not np.isnan(ll)]
if valid_lls:
    m_names, ll_vals = zip(*valid_lls)
    bars = axes[0].bar(m_names, ll_vals, color=colors[:len(m_names)])
    axes[0].set_ylabel('Log-Likelihood ↑')
    axes[0].set_title('Model Fit (higher is better)')
    axes[0].tick_params(axis='x', rotation=45)

# Semantic Cosine
cosines = [results[m].semantic_cosine for m in models]
valid_cos = [(m, c) for m, c in zip(models, cosines) if not np.isnan(c)]
if valid_cos:
    m_names, cos_vals = zip(*valid_cos)
    bars = axes[1].bar(m_names, cos_vals, color=colors[:len(m_names)])
    axes[1].set_ylabel('Cosine Similarity ↑')
    axes[1].set_title('Semantic Prediction (higher is better)')
    axes[1].tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()

## 6️⃣ 波动率分析 (核心功能)

**关键问题**: 能否区分恐慌驱动的事件 vs 趋势驱动的事件？

$$\lambda(t) = \text{Softplus}\left(\underbrace{w_{tr}^\top m(z(t))}_{\text{Trend Channel}} + \underbrace{w_{vol}^\top g(v(t))}_{\text{Volatility Channel}} + \mu_{base}\right)$$

- **Trend-driven events**: 由语义状态z(t)驱动（如：发布疫情数据）
- **Panic-driven events**: 由波动率v(t)驱动（如：谣言引发的恐慌）

In [ ]:
# 分析单个级联
from sv_nsde import VolatilityAnalyzer

analyzer = VolatilityAnalyzer(model, device=device)

# 选择一个测试级联
sample_cascade = test_cascades[0]
sample_embeddings = test_embeddings[sample_cascade.cascade_id]

print(f"\n🔍 Analyzing cascade: {sample_cascade.cascade_id}")
print(f"   Size: {sample_cascade.size} events")
print(f"   Duration: {sample_cascade.duration:.2f} hours")

In [ ]:
# 波动率分解
analysis = analyzer.analyze_cascade(sample_cascade, sample_embeddings)

print(f"\n📊 Volatility Decomposition:")
print(f"   Panic-driven events: {analysis['num_panic_events']}/{sample_cascade.size}")
print(f"   Panic ratio: {analysis['panic_ratio']:.2%}")
print(f"\n   Volatility ratio (first 5 events): {analysis['volatility_ratio'][:5]}")

In [ ]:
# 可视化分解结果
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

times = analysis['event_times']
trend = analysis['trend_contribution']
vol = analysis['volatility_contribution']
vol_ratio = analysis['volatility_ratio']

# 趋势 vs 波动率贡献
ax = axes[0, 0]
ax.plot(times, trend, label='Trend Channel', marker='o', markersize=4)
ax.plot(times, vol, label='Volatility Channel', marker='s', markersize=4)
ax.set_xlabel('Time (hours)')
ax.set_ylabel('Contribution')
ax.set_title('Intensity Decomposition')
ax.legend()
ax.grid(True, alpha=0.3)

# 波动率比例
ax = axes[0, 1]
colors = ['red' if r > 0.5 else 'blue' for r in vol_ratio]
ax.bar(range(len(vol_ratio)), vol_ratio, color=colors, alpha=0.7)
ax.axhline(0.5, color='black', linestyle='--', label='Threshold (0.5)')
ax.set_xlabel('Event Index')
ax.set_ylabel('Volatility Ratio')
ax.set_title('Panic vs Trend Classification')
ax.legend()

# 累积事件
ax = axes[1, 0]
cumulative = np.arange(1, len(times) + 1)
ax.plot(times, cumulative, marker='o', markersize=4)
ax.set_xlabel('Time (hours)')
ax.set_ylabel('Cumulative Events')
ax.set_title('Cascade Growth Pattern')
ax.grid(True, alpha=0.3)

# 事件类型饼图
ax = axes[1, 1]
n_panic = analysis['num_panic_events']
n_trend = len(times) - n_panic
ax.pie([n_trend, n_panic], labels=['Trend-driven', 'Panic-driven'], 
       colors=['#3498db', '#e74c3c'], autopct='%1.1f%%', startangle=90)
ax.set_title('Event Type Distribution')

plt.tight_layout()
plt.show()

In [ ]:
# 分析所有测试级联
print("\n📊 Analyzing all test cascades...")

all_panic_ratios = []
for cascade in tqdm(test_cascades[:50]):
    if cascade.cascade_id not in test_embeddings:
        continue
    try:
        analysis = analyzer.analyze_cascade(
            cascade, 
            test_embeddings[cascade.cascade_id]
        )
        all_panic_ratios.append({
            'cascade_id': cascade.cascade_id,
            'size': cascade.size,
            'panic_ratio': analysis['panic_ratio'],
            'num_panic': analysis['num_panic_events'],
        })
    except:
        pass

df_panic = pd.DataFrame(all_panic_ratios)
print(f"\n📈 Panic Analysis Summary:")
print(f"   Mean panic ratio: {df_panic['panic_ratio'].mean():.2%}")
print(f"   Std panic ratio: {df_panic['panic_ratio'].std():.2%}")
print(f"   High-panic cascades (>50%): {(df_panic['panic_ratio'] > 0.5).sum()}/{len(df_panic)}")

In [ ]:
# 可视化恐慌比例分布
fig, axes = plt.subplots(1, 2, figsize=(12, 4))

# 直方图
axes[0].hist(df_panic['panic_ratio'], bins=20, edgecolor='white', alpha=0.7)
axes[0].axvline(0.5, color='red', linestyle='--', label='Threshold')
axes[0].set_xlabel('Panic Ratio')
axes[0].set_ylabel('Number of Cascades')
axes[0].set_title('Distribution of Panic Ratios')
axes[0].legend()

# 散点图: 级联大小 vs 恐慌比例
axes[1].scatter(df_panic['size'], df_panic['panic_ratio'], alpha=0.6)
axes[1].set_xlabel('Cascade Size')
axes[1].set_ylabel('Panic Ratio')
axes[1].set_title('Size vs Panic Ratio')
axes[1].axhline(0.5, color='red', linestyle='--', alpha=0.5)

plt.tight_layout()
plt.show()

## 7️⃣ 保存模型

In [ ]:
# 保存模型权重
os.makedirs("checkpoints", exist_ok=True)

torch.save({
    'model_state_dict': model.state_dict(),
    'config': {
        'd_latent': D_LATENT,
        'd_hidden': D_HIDDEN,
    },
    'train_losses': train_losses,
    'val_losses': val_losses,
}, 'checkpoints/sv_nsde_model.pt')

print("✓ Model saved to checkpoints/sv_nsde_model.pt")

In [ ]:
# 加载模型示例
checkpoint = torch.load('checkpoints/sv_nsde_model.pt', map_location=device)

loaded_model = SVNSDELite(
    d_input=checkpoint['config']['d_latent'],
    d_latent=checkpoint['config']['d_latent'],
    d_hidden=checkpoint['config']['d_hidden'],
).to(device)

loaded_model.load_state_dict(checkpoint['model_state_dict'])
loaded_model.eval()

print("✓ Model loaded successfully!")

---

## 📚 总结

### 本 Notebook 完成了:

1. ✅ **环境设置** - 安装依赖并检测GPU
2. ✅ **数据准备** - 生成合成数据并构建级联
3. ✅ **模型架构** - 理解SV-NSDE的核心组件
4. ✅ **训练模型** - 完整的训练循环
5. ✅ **评估对比** - 与4个基线模型比较
6. ✅ **波动率分析** - 识别恐慌驱动的事件
7. ✅ **保存模型** - 导出训练好的模型

### 下一步:

- 🔗 下载真实 Weibo-COV 数据: https://github.com/nghuyong/weibo-cov
- 🔧 使用 RoBERTa 编码真实微博文本
- 📈 增加训练 epochs 获得更好性能
- 🧪 进行消融实验验证波动率通道的作用

---

**参考论文**: Chen, Z. (2026). Modeling Crisis Dynamics with Volatility-Modulated Neural SDEs.